In [7]:
#data loading
import pandas as pd

df = pd.read_csv('5transfers_rund.csv')

In [8]:
#-----------------------------------------------create columns----------------------------------------------------------------#

#time difference between two consecutive ocurrences of features in the dataset
def time_diff_consecutive_var(df,col):
    df[col+"_time_diff"] = df.groupby(col)["timestamp"].diff().dt.total_seconds()
    df[col + "_time_diff"] = df[col + "_time_diff"].fillna(-1)
    return df[col+"_time_diff"]

#grouping a feature by same clientid and checking how often is during a selected timewindow
def func_group_clientid_var(df, var, window):
    newcol = df.groupby(["clientid", var], as_index=False)[var].rolling(window).count()
    dff = pd.DataFrame(newcol)
    dff.columns = ["clientid_" + var + "_" + window]
    dff.reset_index(drop = False, inplace=True)
    df.reset_index(drop = 'timestamp' in df.columns, inplace=True)
    dff["clientid_" + var + "_timestamp"] = dff["clientid"].astype(str) + dff[var].astype(str) + dff["timestamp"].astype(str)
    dff.drop_duplicates(subset=["clientid_" + var + "_timestamp"], inplace=True)
    dff.drop("clientid_" + var + "_timestamp", axis = 1, inplace = True)
    df = df.merge(dff, on=["clientid", var, "timestamp"], how="left")
    return df["clientid_" + var + "_" + window]

#average of the amount for a selected feature during a specifice timewindow
def mean_amount_var_time(df: pd.DataFrame, variable: str, timewindow: str):
    df["mean_amount_" + variable + "_" + timewindow] = df.groupby(variable)["amount"].transform(lambda s: s.rolling(timewindow).mean())
    return df["mean_amount_" + variable + "_" + timewindow]

#consecutive fraud ocurrences for the same clientid
def func_fraud_cons_clientid(df):
    df["clientid_is_fraud"] = df["clientid"].astype(str) + df["is_fraud"].astype(str)
    df["is_fraud_cluster"] = (df["clientid_is_fraud"].shift(1) != df["clientid_is_fraud"]).astype(int).cumsum()
    indexes = list(df.drop_duplicates(subset=["is_fraud_cluster"], keep="first", inplace=False).index)
    df["is_fraud_cons"] = df["is_fraud"]
    df.loc[df["is_fraud"] == 1, "is_fraud_cons"] = np.where(df.loc[df["is_fraud"] == 1].index.isin(indexes), 1, 2)
    return df["is_fraud_cons"]

def variables_count_time(df, var1, var2, window):
    df["#" + var1 + "_" + var2 + "_" + window] = df.groupby(var1)[var2].transform(lambda s: s.rolling(window).count())
    df["#" + var1 + "_" + var2 + "_" + window] = df.groupby(var1)["#" + var1 + "_" + var2 + "_" + window].shift(1).fillna(0)
    return df["#" + var1 + "_" + var2 + "_" + window]

In [9]:
##clientid_time_diff - difference in seconds between the last transaction made by the client and the current transaction
df["clientid_time_diff"] = time_diff_consecutive_var(df[["timestamp", "clientid"]].copy(), "clientid")


##iban_dest_time_diff - difference in seconds between the last transaction received by that iban 
df["iban_dest_time_diff"] = time_diff_consecutive_var(df[["timestamp", "iban_dest"]].copy(), "iban_dest")


#clientid_ipadress_30D – number of transactions made by the client with that ip in the 30 days previous to the current transact
df["clientid_ipaddress_30D"] = func_group_clientid_var(df[["timestamp", "clientid", "ipaddress"]].copy(), "ipaddress", "30D")


#clientid_iban_orig_30D - number of transactions made by the client with that iban in the 30 days previous to the current tran
df["clientid_iban_orig_30D"] = func_group_clientid_var(df[["timestamp", "clientid", "iban_orig"]].copy(), "iban_orig", "30D")


#clientid_iban_dest_30D - number of transactions made by the client to that iban in the 30 days previous to the current transac
df["clientid_iban_dest_30D"] = func_group_clientid_var(df[["timestamp", "clientid", "iban_dest"]].copy(), "iban_dest", "30D")


#mean_amount_clientid_30D – mean of the transaction amounts made by that client in the 30 days previous to the current transacac
df["mean_amount_clientid_30D"] = mean_amount_var_time(df[["timestamp", "amount", "clientid"]].copy(), "clientid", "30D")


#is_fraud_cons - identifies consecutive frauds 
df["is_fraud_cons"] = func_fraud_cons_clientid(df[["timestamp", "clientid", "is_fraud"]].copy())

#amount_ratio_avg_30D_per_clientid - current transaction amount divided by the mean of the transactions amounts made by a client
#in the previous 30 days 
df["amount_ratio_avg_clientid_30D"] = df["mean_amount_clientid_30D"] / df["amount"]


#amount_ratio_max_30D_per_clientid - current transaction amount divided by the maximum transactions value made by a client in
#the previous 30 days
col = f"amount_ratio_max_{time_window}_per_clientid"
df[col] = df.groupby("clientid")["accountbalance"].transform(lambda s: s.rolling(time_window).max())
df[col] = df[col] / df["amount"]

#amount_over_account_balance – amount of current transaction divided by the value in the client account 
df["amount_over_account_balance"] = df["amount"] / df["accountbalance"]


#cons_freq_clientid - consecutive transactions where the client is the same 
df["cons_freq_clientid"] = df["clientid"].groupby((df["clientid"] != df["clientid"].shift()).cumsum()).cumcount() + 1


#cons_freq_ipaddress - number of consecutive transactions where the ip is the same 
df["cons_freq_ipaddress"] = df["ipaddress"].groupby((df["ipaddress"] != df["ipaddress"].shift()).cumsum()).cumcount() + 1


#clientid_entity - how many times the client used that entity 
df["#clientid_entity"] = variables_count_time(df[["timestamp", "clientid", "entity"]].copy(), "clientid", "entity", "30D")


#clientid_reference - how many times the client used that reference
df["#clientid_reference"] = variables_count_time(df[["timestamp", "clientid", "reference"]].copy(), "clientid", "reference", "30D")


#amount_ratio_avg_30D_per_entity - current transaction amount divided by the mean of the transactions amount received by a 
#entity in the previous 30 days
col = "amount_ratio_avg_30D_per_entity"
df[col] = df.groupby("entity")["accountbalance"].transform(lambda s: s.rolling("30D").mean())
df[col] = df[col] / df["amount"]


#amount_ratio_max_30D_per_entity - current transaction amount divided by the maximum transactions value received by a entity in 
#the previous 30 days 
col = "amount_ratio_max_30D_per_entity"
df[col] = df.groupby("entity")["accountbalance"].transform(lambda s: s.rolling("30D").max())
df[col] = df[col] / df["amount"]


#amount_ratio_avg_30D_per_reference - current transaction amount divided by the mean of the transactions amount received by a 
#reference in the previous 30 days 
col = "amount_ratio_avg_30D_per_reference"
df[col] = df.groupby("reference")["accountbalance"].transform(lambda s: s.rolling("30D").mean())
df[col] = df[col] / df["amount"]


#amount_ratio_max_30D_per_reference - current transaction amount divided by the maximum transactions value received by a 
#reference in the previous 30 days 
col = "amount_ratio_max_30D_per_reference"
df[col] = df.groupby("reference")["accountbalance"].transform(lambda s: s.rolling("30D").max())
df[col] = df[col] / df["amount"]
 
    
#reference_time_diff - difference in seconds between the current and last transactions made to that reference 
df["reference_time_diff"] = df.groupby("reference")["timestamp"].diff().dt.total_seconds()
df["reference_time_diff"] = np.round(df["reference_time_diff"].fillna(-1) / 100000)  #lower cardinality by binning


#entity_time_diff - difference in seconds between the current and last transactions made to that entity
df["entity_time_diff"] = df.groupby("entity")["timestamp"].diff().dt.total_seconds()
df["entity_time_diff"] = np.round(df["entity_time_diff"].fillna(-1) / 100000)  #lower cardinality by binning

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [ ]:
#profiling of the balanced data after the columns creation
from pandas_profiling import ProfileReport

#ProfileReport(predata.sample(n = 10000)) for much data - it randomizes data order and selects representative sample
prof = ProfileReport(df, minimal = True)             
prof.to_file(output_file = '5transfers_balanced_smote+rund_allc.html')

In [ ]:
#drop the columns that didn't pass the correlation tests
df.drop(['clientid_30D'], axis = 1, inplace = True)

In [ ]:
#make sure the right columns are saved in the new dataset
df.columns

In [ ]:
#save new dataset
df.to_csv('5transfers_balanced_gan+rund_allc.csv')